In [1]:
import numpy as np
import sympy as sp

def analyze_directions(f_expr, constraints, x0, directions, maximize=False):
    x_vars = sorted(f_expr.free_symbols, key=lambda s: s.name)
    x_subs = dict(zip(x_vars, x0))

    # Шаг 1: найти активные ограничения
    active_constraints = []
    for g in constraints:
        g_val = g.evalf(subs=x_subs)
        if np.isclose(g_val, 0.0, atol=1e-6): 
            active_constraints.append(g)

    print("Активные ограничения:")
    for g in active_constraints:
        print("  ", g)

    # Шаг 2: посчитать градиенты
    grad_f = sp.Matrix([sp.diff(f_expr, var) for var in x_vars])
    grad_constraints = [
        sp.Matrix([sp.diff(g, var) for var in x_vars]) for g in active_constraints
    ]

    # Шаг 3: проверка направлений
    feasible_directions = []

    print("\nПроверка направлений:")
    for idx, s in enumerate(directions):
        s_vec = np.array(s)
        print(f"Направление s{idx+1} = {s_vec}")

        is_feasible = True
        for grad_g in grad_constraints:
            dot_product = float(grad_g.evalf(subs=x_subs).dot(sp.Matrix(s_vec)))
            if dot_product > 0:  # <grad(g_i), s> < 0 – иначе нарушается ограничение
                is_feasible = False
                break

        if is_feasible:
            feasible_directions.append((idx, s_vec))
            print("  -> допустимое")

            dot_grad_f = float(grad_f.evalf(subs=x_subs).dot(sp.Matrix(s_vec)))
            if (not maximize and dot_grad_f < 0) or (maximize and dot_grad_f > 0):
                print("  -> улучшает целевую функцию")
            else:
                print("  -> не улучшает целевую функцию")
        else:
            print("  -> не является допустимым")

# Пример использования:
x1, x2, x3 = sp.symbols('x1 x2 x3')
f = -1/2*x1**2 - x2**2 - 3*x3**2 + 2*x2*x3 - 4
constraints = [
    3/2*x1**2 + 4*x2**2 + 5/2*x3**2 + 2*x1*x3 - 7,
    2*x1**2 + 3*x2**2 + 1/2*x3**2 + 2*x1*x2 - 3,
    2*x1 + x2 - 2*x3 - 2,
    -x1,
    -x3
    ]

x0 = [1, -1, 0]
directions = [
    np.array([0, 1, 1]),
    np.array([-1, 0, -1]),
    np.array([-1, 1, 1]),
]

directions += [np.random.rand(3) for _ in range(10)]

analyze_directions(f, constraints, x0, directions, maximize=True)


ModuleNotFoundError: No module named 'sympy'